<a href="https://colab.research.google.com/github/eygpcr/hydrophobic_interactions/blob/main/hydrophobic_occupancy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hydrophobic Interaction Occupancy Calculation

This notebook identifies hydrophobic interactions between ligand and receptor residues, calculates their occupancy over the course of a molecular dynamics simulation, and visualizes the results through a histogram.

In [ ]:
# Original code from the uploaded notebook